In [4]:
import numpy as np
import pandas as pd
import math
import random
import copy

In [5]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    from zipfile import ZipFile
    
    response = urllib.request.urlretrieve(
        url,filename)
    #unzip
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()

In [6]:
# define fuction to read data from xml file
def import_data(filename):
    import xml.etree.ElementTree as et 
    xtree = et.parse(filename)
    xroot = xtree.getroot()
    
    return xroot;

In [7]:
# define function to create distance matrix
def dist_matrix(cities, xroot):
    #create distance matrix
    import numpy as np
    distance = np.zeros((cities,cities))
    
    #import data
    import xml.etree.ElementTree as et
    from_node = 0
    for child in xroot.iter('vertex'):
        for child1 in child:
            dist = float(child1.attrib.get('cost'))
            to_node = int(child1.text)
            distance[from_node, to_node] = dist

        from_node += 1
    
#     max_distance = np.nanmax(distance)
#     for i in range(cities):#
#         distance[i,i] = max_distance*10 #very large number for distance to itself => no revisited 

    return distance 

In [ ]:
# Medium size data set: # of nodes = 152
url1 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr152.xml.zip'
zip1 = 'pr152.xml.zip'
file1 = 'pr152.xml'
cities1 = 152

load_data(url1, zip1)
xroot1 = import_data(file1)
distance1 = dist_matrix(cities1, xroot1)

In [26]:
# function to calculate priori attractiveness of a decision 
# desirablity /visibility = inverse of the distance between two nodes (local information)
def visibility (distance):
    distance = np.array(distance, dtype = np.float64)
    for i in range(distance.shape[0]):
        for j in range(distance.shape[1]):
            if distance[i,j] != 0:
                distance[i,j] = 1.0/distance[i,j]
                
    return distance

In [41]:
#define class Agent (ant) and its function
class Agent:
    import numpy as np
    import pandas as pd
    import math
    
    def __intit__(self, k, cities): #initialization
        self.index = k
        self.currCity = 0 #start at depot
        self.visited = [0]
        self.cost = 0
        self.unvisited = [x for x in range(1,cities+1)] 
        self.transition_probs = []
        self.tour = []
    
    def reset_agent(self): #reset whenever starting new iteration
        self.currCity = 0 
        self.visited = [0]
        self.cost = 0
        self.unvisited = [x for x in range(1,cities+1)]
        self.transition_probs = []
        self.tour = []

#     def city_sum(self, tau_list, vis_list, i, j): #return tau_ij^alpha * (1/d_ij)^beta
#         tau_part = math.pow(tau_list[i,j], alpha)
#         vis_part = math.pow(vis_list[i,j], beta)
#         return tau_part*vis_part
        
    # get current transition prob (of unvisited city)
    def get_trans_prob(self,tau_list, vis_list, j): #j in unvisited
        tau_part = math.pow(tau_list[i,j], alpha)
        vis_part = math.pow(vis_list[i,j], beta)
        
        #numerator = self.city_sum(tau_list, vis_list, self.currCity, j)
        #denominator = sum( [ self.city_sum(tau_list, vis_list, self.currCity, l) for l in self.unvisited] )
        
        return tau_part*vis_part
        
    
#     # update new transition prob 
#     def cal_trans_prob(self,tau_list, vis_list, j): #return p_ijk
#         numerator = self.city_sum(tau_list, vis_list, self.currCity, j) #j in unvisited list
#         denominator = sum( [ self.city_sum(tau_list, vis_list, self.currCity, l) for l in self.unvisited] )

#         return numerator/denominator
    

#     def move(self, distance, j): #move from current position to j
#         self.cost += distance[self.currCity, j] #update cost
#         self.currCity = j #update current position 
#         self.visited.append(j) #update visited list
#         #if j in self.unvisited.remove: 
#         self.unvisited.remove(j)
#         #self.transition_prob = []
    

    def move(self, distance, tau_list, vis_list):
        self.reset_agent()
        
        while (self.unvisited):
            for j in self.unvisited:
                self.transition_probs.append(self.get_trans_prob(tau_list, vis_list, j))            

            self.transition_probs= self.transition_probs/sum(self.transition_probs) #normalize
            nextCity = numpy.random.choice(self.unvisited,1,p=self.transition_probs) #random select next city based on trans_prob
            
            self.tour.append((self.currCity, nextCity))
            self.visited.append(nextCity) #update into visited
            self.unvisited.remove(nextCity) #remove from unvisited
            self.cost += distance[self.currCity, nextCity] #update length of the tour
            
            self.currCity=nextCity #update current position
            self.transition_probs=[] #reset transition prob


In [39]:
iterations = 50 #number of iterations

# intensities 
intensity = 0.0001

# # of ants: default value = # of cities
# too many ants will reinforce not optimal solutions (visit un-optimal tour too often)
# too few ants would not produce cooperative effect due to pheromone decay
agents = cities #number of agents

# Relative weight of feedback: how greedy the algorithm is => EXPLOITATION EFFECT 
# alpha = 0 => extremly greedy (alpha >= 0) => control the influence of the amount of pheromone deposited 
alpha = 1

# Relative weight of problem information: how fast the ants is converge to a steady solution => EXPLORATION EFFECT
# beta >= 1 => control desirability of state transition (i.e. 1/d)
beta = 2

# Trail increment: how much pheromone is deposited in between steps in the process
# ontrol the level of exploration undertaken by the ants
Q = 1

# Pheromone evaporation rate (Forgetting rate)
# small = slow evaporation => more amplification of the initial random fluctuation
# high = fast evaporation => evaporate quickly (less influence of memory on decision) => constantly doing random searches
rho = 0.1

In [9]:
# 120 cities
cities = 120

di = pd.read_excel("gr120.xlsx",sheet_name="DistanceMatrix")
distance=di.values

#for i in range(cities):
#    distance[i,i] = 10000

ds = visibility(distance)
taus = intensity * np.ones(distance.shape)

In [ ]:
#main algorithm
#run iterations
for it in range(iterations):
    colony = []
    for k in range(agents):
        colony.append(Ant(k, cities))
    
    delta_tau = np.zeros((agents, cities+1, cities+1))
    for k in range(agents):
        colony[k].move(distance, taus, ds)
        tour_k = colony[k].visited
        L_k = colony[k].cost
        for i in range(cities+1):
            for j in range(cities+1):
                if (i,j) is in colony[k].tour:
                    delta_tau[k,i,j] = Q/L_k
    for i in range(cities+1):
        for j in range(cities+1):
            taus[i,j] = rho*taus[i,j] + sum( [delta_tau[k,i,j] for k in range(agents)] )
            



In [27]:
visibility(distance)

array([[0.        , 0.00187266, 0.00230415, ..., 0.00229885, 0.00271003,
        0.00826446],
       [0.00187266, 0.        , 0.00934579, ..., 0.00153139, 0.00598802,
        0.00195695],
       [0.00230415, 0.00934579, 0.        , ..., 0.00178571, 0.01265823,
        0.00239234],
       ...,
       [0.00229885, 0.00153139, 0.00178571, ..., 0.        , 0.00204918,
        0.003125  ],
       [0.00271003, 0.00598802, 0.01265823, ..., 0.00204918, 0.        ,
        0.00288184],
       [0.00826446, 0.00195695, 0.00239234, ..., 0.003125  , 0.00288184,
        0.        ]])

In [25]:
distance


array([[  0, 534, 434, ..., 435, 369, 121],
       [534,   0, 107, ..., 653, 167, 511],
       [434, 107,   0, ..., 560,  79, 418],
       ...,
       [435, 653, 560, ...,   0, 488, 320],
       [369, 167,  79, ..., 488,   0, 347],
       [121, 511, 418, ..., 320, 347,   0]], dtype=int64)

None
